In [1]:
import numpy as np
from collections import Counter, defaultdict

class KmerProcessor:
    """
    A foundational class for k-mer processing and frequency simulation.
    Simulates the global k-mer frequency query functionality provided by the Count-Min Sketch in the pipeline.
    """
    def __init__(self, k=13):
        """
        Initializes the processor.
        k=13 is used here for the short example sequences. For long reads, k=21 is typical.
        """
        self.k = k
        # Using a precise dictionary to simulate CMS's approximate counting.
        self.kmer_freq = Counter()
        self.base_counter = Counter()

    def process_sequence(self, sequence, allow_ambiguous=False):
        """
        Processes a sequence, extracts all definitive k-mers, and updates frequencies.
        :param sequence: Input sequence string.
        :param allow_ambiguous: Whether to allow ambiguous characters in the k-mer.
        """
        sequence = sequence.upper()
        for i in range(len(sequence) - self.k + 1):
            kmer = sequence[i:i + self.k]
            if not allow_ambiguous and ('N' in kmer or '?' in kmer):
                continue
            self.kmer_freq[kmer] += 1
            for base in kmer:
                self.base_counter[base] += 1

    def get_kmer_frequency(self, kmer):
        """Queries the (approximate) frequency of a given k-mer."""
        return self.kmer_freq.get(kmer, 0)

    def get_base_background(self, base):
        """Gets the background frequency of a single base."""
        total = sum(self.base_counter.values())
        if total == 0:
            return 0.25
        return self.base_counter.get(base, 0) / total


class ProbabilisticReconstructor:
    """
    Core class for probabilistic reconstruction.
    Implements Bayesian inference based on local context and global k-mer spectrum.
    """
    def __init__(self, kmer_processor, pseudo_count=0.1):
        """
        Initializes the reconstructor.
        :param kmer_processor: An object providing k-mer frequency queries.
        :param pseudo_count: Pseudocount for probability smoothing.
        """
        self.kp = kmer_processor
        self.pseudo_count = pseudo_count
        self.bases = ['A', 'T', 'C', 'G']

    def find_similar_fragments(self, target_seq, fragment_list, threshold=0.5):
        """
        Finds fragments similar to the target based on simple identity (simplified for example).
        In the actual pipeline, this is done via fast MinHash signature comparison.
        """
        similar = []
        target_len = len(target_seq)
        for frag in fragment_list:
            if frag == target_seq:
                continue
            match = sum(1 for a, b in zip(target_seq, frag) if a == b and a not in 'N?')
            if match / target_len > threshold:
                similar.append(frag)
        return similar

    def reconstruct_position(self, target_seq, pos, similar_fragments):
        """
        Reconstructs the ambiguous base at a specific position in the target sequence.
        """
        # Step 1 & 2: Calculate Prior and Likelihood
        prior_probs = {}
        likelihood_probs = {base: self.pseudo_count for base in self.bases}

        # Calculate prior based on global k-mer frequency
        for base in self.bases:
            candidate_kmer = self._construct_candidate_kmer(target_seq, pos, base)
            freq = self.kp.get_kmer_frequency(candidate_kmer) + 1e-6
            prior_probs[base] = freq

        # Normalize prior
        total_prior = sum(prior_probs.values())
        prior_probs = {b: p / total_prior for b, p in prior_probs.items()}

        # Calculate likelihood based on similar fragments
        if similar_fragments:
            base_counts = Counter()
            for frag in similar_fragments:
                if pos < len(frag) and frag[pos] in self.bases:
                    base_counts[frag[pos]] += 1
            total_counts = sum(base_counts.values())
            if total_counts > 0:
                for base, count in base_counts.items():
                    likelihood_probs[base] = count / total_counts

        # Step 3: Bayesian Inference - Calculate Posterior
        unnormalized_posterior = {}
        for base in self.bases:
            unnormalized_posterior[base] = likelihood_probs[base] * prior_probs[base]
        total_posterior = sum(unnormalized_posterior.values())
        posterior_probs = {b: p / total_posterior for b, p in unnormalized_posterior.items()}

        # Step 4: Decision
        reconstructed_base = max(posterior_probs.items(), key=lambda x: x[1])[0]
        return reconstructed_base, posterior_probs, prior_probs, likelihood_probs

    def _construct_candidate_kmer(self, sequence, pos, candidate_base):
        """Constructs a k-mer centered around pos with the candidate base."""
        half_k = self.kp.k // 2
        start = max(0, pos - half_k)
        end = start + self.kp.k
        if end > len(sequence):
            end = len(sequence)
            start = end - self.kp.k
        seq_list = list(sequence[start:end])
        seq_list[pos - start] = candidate_base
        return ''.join(seq_list)


def main():
    """Main function to demonstrate the complete reconstruction workflow."""
    fragments = [
        "ATGCC??TAGGCTATANNCTG",
        "ATGACCTAGG??ATTACACTG",
        "GTGCCA?AGGCTATACNCT?"
    ]

    print("=" * 60)
    print("Demonstration: Probabilistic Reconstruction Module")
    print("=" * 60)

    # Step 1: Build k-mer frequency background (simulating upstream pipeline)
    print("\n1. Building k-mer frequency background...")
    kp = KmerProcessor(k=13)
    for frag in fragments:
        kp.process_sequence(frag, allow_ambiguous=False)

    # Step 2 & 3: Reconstruct each fragment
    reconstructor = ProbabilisticReconstructor(kp)
    all_results = []
    for idx, frag in enumerate(fragments):
        print(f"\n{'='*60}")
        print(f"Reconstructing Fragment {chr(65+idx)}: {frag}")
        print(f"{'='*60}")
        similar = reconstructor.find_similar_fragments(frag, fragments, 0.4)
        reconstructed_seq = list(frag)
        details = []
        ambiguous_positions = [i for i, b in enumerate(frag) if b in 'N?']
        for pos in ambiguous_positions:
            best_base, posterior, prior, likelihood = reconstructor.reconstruct_position(frag, pos, similar)
            reconstructed_seq[pos] = best_base
            details.append({
                'position': pos,
                'reconstructed': best_base,
                'posterior': posterior,
                'prior': prior,
                'likelihood': likelihood
            })
            # Print inference details for this position
            ctx_start = max(0, pos-3)
            ctx_end = min(len(frag), pos+4)
            context = frag[ctx_start:pos] + '[' + frag[pos] + ']' + frag[pos+1:ctx_end]
            print(f"  Position {pos} (Context: {context}):")
            print(f"    Prior: {prior}")
            print(f"    Likelihood: {likelihood}")
            print(f"    Posterior: {posterior}")
            print(f"    Decision: '{frag[pos]}' -> '{best_base}' (Prob: {posterior[best_base]:.3f})")
        final_seq = ''.join(reconstructed_seq)
        all_results.append({'original': frag, 'reconstructed': final_seq, 'details': details})
        print(f"\n  Fragment {chr(65+idx)} Result:")
        print(f"    Original:  {frag}")
        print(f"    Reconstructed: {final_seq}")

    # Summary
    print(f"\n{'='*60}")
    print("Reconstruction Summary")
    print(f"{'='*60}")
    for idx, res in enumerate(all_results):
        changes = [(i, res['original'][i], res['reconstructed'][i]) for i in range(len(res['original'])) if res['original'][i] in 'N?' and res['original'][i] != res['reconstructed'][i]]
        print(f"\nFragment {chr(65+idx)}: {len(changes)} base(s) reconstructed.")
        for pos, old, new in changes:
            prob = next(d['posterior'][new] for d in res['details'] if d['position'] == pos)
            print(f"  Pos {pos}: {old} -> {new} (Posterior Prob: {prob:.3f})")


if __name__ == "__main__":
    main()

Demonstration: Probabilistic Reconstruction Module

1. Building k-mer frequency background...

Reconstructing Fragment A: ATGCC??TAGGCTATANNCTG
  Position 5 (Context: GCC[?]?TA):
    Prior: {'A': 0.25, 'T': 0.25, 'C': 0.25, 'G': 0.25}
    Likelihood: {'A': 0.1, 'T': 0.1, 'C': 1.0, 'G': 0.1}
    Posterior: {'A': 0.07692307692307693, 'T': 0.07692307692307693, 'C': 0.7692307692307692, 'G': 0.07692307692307693}
    Decision: '?' -> 'C' (Prob: 0.769)
  Position 6 (Context: CC?[?]TAG):
    Prior: {'A': 0.25, 'T': 0.25, 'C': 0.25, 'G': 0.25}
    Likelihood: {'A': 0.1, 'T': 1.0, 'C': 0.1, 'G': 0.1}
    Posterior: {'A': 0.07692307692307693, 'T': 0.7692307692307692, 'C': 0.07692307692307693, 'G': 0.07692307692307693}
    Decision: '?' -> 'T' (Prob: 0.769)
  Position 16 (Context: ATA[N]NCT):
    Prior: {'A': 0.25, 'T': 0.25, 'C': 0.25, 'G': 0.25}
    Likelihood: {'A': 0.1, 'T': 0.1, 'C': 1.0, 'G': 0.1}
    Posterior: {'A': 0.07692307692307693, 'T': 0.07692307692307693, 'C': 0.7692307692307692, 'G